In [8]:
!pip -q install gspread gspread-dataframe beautifulsoup4 lxml httpx

In [9]:
from google.colab import auth
auth.authenticate_user()

import google.auth
import gspread
from gspread_dataframe import set_with_dataframe
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials

# In Colab, default user creds are fine:
creds, _ = google.auth.default(scopes=[
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
])
gc = gspread.authorize(creds)

In [10]:
SELLER_FEEDBACK_URL = "https://www.ebay.co.uk/fdbk/feedback_profile/top-bargains-uk?filter=feedback_page%3ARECEIVED_AS_SELLER&sort=RELEVANCEV2"

# How many feedback pages to scan (start with 3–5; increase if needed)
MAX_PAGES = 10

# Your Google Sheet (created if not found)
SPREADSHEET_NAME = "eBay Feedback — top-bargains-uk"
WORKSHEET_NAME   = "feedback"

# Optional: also save a CSV in Colab for download
SAVE_CSV = True
CSV_PATH = "/content/feedback_top_bargains_uk.csv"


In [11]:
import re, time, hashlib
import httpx
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin, urlparse, parse_qs

HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0 Safari/537.36"
}

def fetch_html(url):
    with httpx.Client(headers=HEADERS, timeout=30.0, follow_redirects=True) as client:
        r = client.get(url)
        r.raise_for_status()
        return r.text

def make_row_id(date_text, rating, comment, item_url):
    key = f"{date_text}|{rating}|{comment[:200]}|{item_url or ''}"
    return hashlib.sha1(key.encode("utf-8")).hexdigest()

def extract_item_url(container):
    # Look for any link that points to an item page
    a = container.select_one("a[href*='/itm/']")
    if a and a.has_attr("href"):
        return a["href"]
    # fallback: sometimes the link is nested in small tags
    for a in container.find_all("a", href=True):
        if "/itm/" in a["href"]:
            return a["href"]
    return None

def clean_text(x):
    return re.sub(r"\s+", " ", x).strip() if x else ""

def parse_rating(container):
    # Many feedback pages have an icon or text indicating Positive/Neutral/Negative
    # Try common patterns:
    text = container.get_text(" ", strip=True).lower()
    # Look for explicit words first:
    for word in ["positive", "neutral", "negative"]:
        if word in text:
            return word.capitalize()
    # Try alt/title attributes on images/icons
    img = container.find("img")
    for attr in ("alt", "title"):
        if img and img.get(attr):
            t = img.get(attr).lower()
            for word in ["positive", "neutral", "negative"]:
                if word in t:
                    return word.capitalize()
    # Fallback: unknown
    return "Unknown"

def parse_date(container):
    # Try a set of selectors commonly used on eBay feedback pages
    for sel in [".fbDate", ".feedback-date", ".date", "span[aria-label*='Feedback left on']"]:
        el = container.select_one(sel)
        if el:
            return clean_text(el.get_text())
    # Fallback: scan for date-like strings (very loose)
    text = container.get_text(" ", strip=True)
    m = re.search(r"(?:\d{1,2}\s+\w+\s+\d{4}|\d{1,2}-\d{1,2}-\d{2,4}|\d{1,2}\s+\w+)", text)
    return m.group(0) if m else ""

def parse_comment(container):
    # Try common selectors for feedback comment
    for sel in [".fbComment", ".comment", ".feedback-text", "p"]:
        el = container.select_one(sel)
        if el and clean_text(el.get_text()):
            return clean_text(el.get_text())
    # Fallback: take the largest text block
    return clean_text(container.get_text())

def parse_feedback_page(html):
    soup = BeautifulSoup(html, "lxml")

    # Feedback rows: try a few container patterns
    candidates = (
        soup.select("div.feedback-item") or
        soup.select("li.feedback-entry") or
        soup.select("div.fbItem") or
        soup.select(".fb-FeedbackList .fbBox") or
        soup.select("div#FeedbackList div")
    )

    rows = []
    for c in candidates:
        comment = parse_comment(c)
        if not comment or len(comment) < 3:
            continue  # skip junk
        date_text = parse_date(c)
        rating    = parse_rating(c)
        item_url  = extract_item_url(c)
        rid       = make_row_id(date_text, rating, comment, item_url)
        rows.append({
            "row_id": rid,
            "date": date_text,
            "rating": rating,
            "comment": comment,
            "item_url": item_url
        })
    return rows

def find_next_page_url(html, current_url):
    soup = BeautifulSoup(html, "lxml")
    # Look for a link element that indicates 'Next'
    # eBay often uses ?_pgn=2 for pagination
    a_next = None
    for a in soup.find_all("a", href=True):
        txt = (a.get_text() or "").strip().lower()
        if "next" in txt:
            a_next = a
            break
    if a_next:
        return urljoin(current_url, a_next["href"])

    # fallback: try to auto-increment ?_pgn=
    parsed = urlparse(current_url)
    qs = parse_qs(parsed.query)
    pgn = None
    for k in qs:
        if "pgn" in k.lower() or "_pgn" in k.lower():
            try:
                pgn = int(qs[k][0])
                next_q = parsed.query.replace(str(pgn), str(pgn+1))
                return parsed._replace(query=next_q).geturl()
            except:
                pass
    return None

In [12]:
all_rows = []
seen = set()

url = SELLER_FEEDBACK_URL
for i in range(MAX_PAGES):
    try:
        html = fetch_html(url)
    except Exception as e:
        print(f"Error fetching page {i+1}: {e}")
        break

    rows = parse_feedback_page(html)
    new = [r for r in rows if r["row_id"] not in seen]
    for r in new:
        seen.add(r["row_id"])
    all_rows.extend(new)

    print(f"Parsed page {i+1}: {len(new)} new rows, total {len(all_rows)}")

    nxt = find_next_page_url(html, url)
    if not nxt:
        break
    url = nxt
    time.sleep(1.0)  # be polite

# Build DataFrame
df = pd.DataFrame(all_rows, columns=["date","rating","comment","item_url","row_id"])
# Keep nice order
df = df[["date","rating","comment","item_url","row_id"]]
print("Total feedback rows collected:", len(df))
df.head(3)

Parsed page 1: 0 new rows, total 0
Total feedback rows collected: 0


,date,rating,comment,item_url,row_id


In [13]:
# Get or create spreadsheet
try:
    sh = gc.open(SPREADSHEET_NAME)
except gspread.SpreadsheetNotFound:
    sh = gc.create(SPREADSHEET_NAME)
    # Share with your Google account for easy access (optional):
    # sh.share('<your_gmail>@gmail.com', perm_type='user', role='writer')

# Get or create worksheet
try:
    ws = sh.worksheet(WORKSHEET_NAME)
    ws.clear()
except gspread.WorksheetNotFound:
    ws = sh.add_worksheet(title=WORKSHEET_NAME, rows=str(max(len(df), 100)), cols="10")

# Write DataFrame
set_with_dataframe(ws, df)

sheet_url = sh.url
print("✅ Written to Google Sheets:", sheet_url)

if SAVE_CSV:
    df.to_csv(CSV_PATH, index=False)
    print("Also saved CSV at:", CSV_PATH)

✅ Written to Google Sheets: https://docs.google.com/spreadsheets/d/1CiK_3u0dBEbMz-XkHHhNVHG7AC8WlRRnBGiriegGE6w
Also saved CSV at: /content/feedback_top_bargains_uk.csv
